In [1]:
%load_ext autoreload
%autoreload 2

from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import scipy as sp
from sklearn import linear_model

from fama_french import load_monthly_factors, ff_display

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = 12, 6

NameError: name 'matplotlib' is not defined

# Betterment Performance Data

Betterment's performance data is located on [their performance page](https://www.betterment.com/resources/betterment-historical-performance/) in an IFrame [that points to CloudFront](https://d1svladlv4b69d.cloudfront.net/src/d3/bmt-hist-perf-line-graph/bmt-hist-perf.html).  We downloaded the HTML of the latter on June 21st.

In [ ]:
with open("data/Betterment_Performance.html") as fh:
    soup = BeautifulSoup(fh)
    paths = soup.select("g.g-linecontainer path.g-port-line")


## SVG Graphics

SVG graphics language is an `M[x],[y]` followed by a series of `L[x],[y]` commands ([source](https://developer.mozilla.org/en-US/docs/Web/SVG/Tutorial/Paths)).  They can be parsed from "path" objects.  Remember that the x coordinates increase left to right but the y coordinates increase *up to down* ([source](https://www.w3.org/TR/SVG/coords.html)), as with most coodinate systems for monitor displays.

In an email, Betterment staff explained that the returns dated from the end of the month.

In [ ]:
# extract Jan 2004 to Feb 2019 (inclusive) values
dates = pd.date_range(start='2004/1/31', end='2019/2/28', freq='M')

def extract_y_coors(data):
    assert(data.startswith('M'))
    pairs = [[float(x) for x in pair.split(',')] for pair in data[1:].split('L')]
    assert(len(pairs) == len(dates))
    return [y for _, y in pairs]

df_y = pd.DataFrame({
    path.get('class')[1]: extract_y_coors(path.get('d'))
    for path in paths
}, index=dates).sort_index()

df_y.tail()


## Parse Returns Table

In [ ]:
def parse_percent(x):
    if isinstance(x, str) and x.endswith('%'):
        return float(x[:-1]) / 100
    return x

with open("data/Betterment_Performance.html") as fh:
    df_raw = pd.read_html(fh)[0]
    df_r = pd.DataFrame(
        df_raw.iloc[2:,:].values,
        columns=df_raw.iloc[1,:].values
    ).set_index('Portfolio').applymap(parse_percent)
    
df_r

# Join path data and Return Table

We're in luck -- the data aligns so it's easy to join them

In [ ]:
label_dict = dict(zip(df_y.columns, df_r.index))
label_dict

In [ ]:
if set(df_y.columns) != set(label_dict.values()):
    df_y.columns = [label_dict[x] for x in df_y.columns]
df_y.tail()

In [ ]:
df_ry = pd.merge(
    df_r, df_y.iloc[-1:, :].T,
    right_index=True,
    left_index=True
)
df_ry

## Connect path coodinates with returns

We use the terminal coordinates for each index and match them (via linear regression) against the cumulative returns to infer the monthly returns.

In [ ]:
# Slightly overdone use of linear regression
lm = linear_model.LinearRegression()
X = df_ry[[pd.Timestamp('2019-02-28 00:00:00', freq='M')]]
y = df_ry['Cumulative Return'] + 1.  # index values

lm.fit(X, y)
print("R^2: {}".format(lm.score(X, y)))

In [ ]:
df_index = df_y.apply(lambda col: lm.predict(col[:, np.newaxis]))
df_index.head()

In [ ]:
df_index.to_csv("data/betterment_values.csv")

## Returns

In [ ]:
df_returns = pd.DataFrame(
    df_index.iloc[1:,:].values / df_index.iloc[:-1,:].values,
    columns = df_index.columns,
    index = df_index.index[1:]
) - 1.
df_returns['Year'] = df_returns.index.year
df_returns['Month'] = df_returns.index.month
df_returns.head()

## Betterment performs 0.02% worse than S&P 500

In [ ]:
(df_returns['Betterment 100% stock'] - df_returns['S&P 500 Index']).mean() * 12

In [ ]:
df_annual_returns = ((df_returns + 1.)
    .drop(['Year', 'Month'], axis=1)
    .groupby(lambda x: x.year)
    .prod() - 1.)
df_annual_returns.to_csv('data/betterment_annual_returns.csv')
df_annual_returns.loc[[2012, 2013, 2014, 2015, 2016, 2017, 2018]]

## Load Monthly Fama French Factors

In [ ]:
df_factors = load_monthly_factors()
df_factors.head()


In [ ]:
df = df_returns.merge(
    df_factors,
    on=('Year', 'Month'),
    how='inner'
)

df.head()

# Perhaps slighty worse with Fama French

In [ ]:
index_cols = [
   'S&P 500 Index', 'Betterment 100% stock',
   'Betterment 80% stock', 'Betterment 70% stock', 'Betterment 60% stock',
   'Betterment 50% stock',
   'Avg. private client investor 80-100% equity risk',
   'Betterment 40% stock',
   'Avg. private client investor 60-80% equity risk',
   'Betterment 30% stock',
   'Avg. private client investor 40-60% equity risk',
   'Betterment 20% stock',
   'Avg. private client investor 0-40% equity risk',
   'Five year U.S. Treasury Bills', 'Betterment 10% stock',
   'Betterment 0% stock'
]

waterfall_cols = [
    'S&P 500 Index', 'Betterment 100% stock',
    'Betterment 80% stock'
]

ff_display(df, index_cols, waterfall_cols, monthly=True)